In [1]:
import pandas as pd
import numpy as np

In [2]:
dataPreprocessed = pd.read_csv('Absenteeism_preprocessedSelf.csv')

In [3]:
dataPreprocessed.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


In [4]:
dataPreprocessed['Absenteeism Time in Hours'].median()

3.0

In [5]:
targets = np.where(dataPreprocessed['Absenteeism Time in Hours'] > dataPreprocessed['Absenteeism Time in Hours'].median(),
                   1, 0)

In [6]:
dataPreprocessed['Excessive Absenteeism'] = targets

In [7]:
dataPreprocessed.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0


In [8]:
targets.sum() / targets.shape[0]

0.45571428571428574

In [9]:
dataWithTargets = dataPreprocessed.drop(['Absenteeism Time in Hours', 'Day of the Week', 'Daily Work Load Average', 'Distance to Work'], axis=1)

In [10]:
dataWithTargets.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,0,0,0,1,7,289,33,30,0,2,1,1
1,0,0,0,0,7,118,50,31,0,1,0,0
2,0,0,0,1,7,179,38,31,0,0,0,0
3,1,0,0,0,7,279,39,24,0,2,0,1
4,0,0,0,1,7,289,33,30,0,2,1,0


In [11]:
unscaledInputs = dataWithTargets.iloc[:,:-1]

In [12]:
#from sklearn.preprocessing import StandardScaler
#absenteeismScaler = StandardScaler()

In [13]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class CustomScaler(BaseEstimator,TransformerMixin):

    def __init__(self,columns):
        self.scaler = StandardScaler()
        self.columns = columns
        self.mean_ = None
        self.var_ = None

    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns],y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self

    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [14]:
unscaledInputs.columns.values

array(['Reason 1', 'Reason 2', 'Reason 3', 'Reason 4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [15]:
columnsToOmit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Education']

In [16]:
columnsToScale = [x for x in unscaledInputs if x not in columnsToOmit]

In [17]:
absenteeismScaler = CustomScaler(columnsToScale)

In [18]:
absenteeismScaler.fit(unscaledInputs)

C:\Users\gjpet\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3785: FutureWarning: The behavior of DataFrame.var with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return var(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


CustomScaler(columns=['Reason 1', 'Reason 2', 'Reason 3', 'Reason 4',
                      'Month Value', 'Transportation Expense', 'Age',
                      'Body Mass Index', 'Children', 'Pets'])

In [19]:
scaledInputs = absenteeismScaler.transform(unscaledInputs)

In [20]:
scaledInputs

,Reason 1,Reason 2,Reason 3,Reason 4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,-0.577350,-0.092981,-0.314485,0.821365,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
1,-0.577350,-0.092981,-0.314485,-1.217485,0.182726,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690
2,-0.577350,-0.092981,-0.314485,0.821365,0.182726,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
3,1.732051,-0.092981,-0.314485,-1.217485,0.182726,0.854936,0.405184,-0.643782,0,0.880469,-0.589690
4,-0.577350,-0.092981,-0.314485,0.821365,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...
695,1.732051,-0.092981,-0.314485,-1.217485,-0.388293,-0.654143,0.562059,-1.114186,1,0.880469,-0.589690
696,1.732051,-0.092981,-0.314485,-1.217485,-0.388293,0.040034,-1.320435,-0.643782,0,-0.019280,1.126663
697,1.732051,-0.092981,-0.314485,-1.217485,-0.388293,1.624567,-1.320435,-0.408580,1,-0.919030,-0.589690
698,-0.577350,-0.092981,-0.314485,0.821365,-0.388293,0.190942,-0.692937,-0.408580,1,-0.919030,-0.589690


In [21]:
scaledInputs.shape

(700, 11)

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
xTrain, xTest, yTrain, yTest = train_test_split(scaledInputs, targets, train_size = 0.8, random_state = 10)

In [24]:
print(xTrain.shape, yTrain.shape)

(560, 11) (560,)


In [25]:
print(xTest.shape, yTest.shape)

(140, 11) (140,)


In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [27]:
reg = LogisticRegression()

In [28]:
reg.fit(xTrain, yTrain)

LogisticRegression()

In [29]:
reg.score(xTrain, yTrain)

0.7875

In [30]:
modelOutputs = reg.predict(xTrain)

In [31]:
np.sum((modelOutputs == yTrain)) / modelOutputs.shape[0]

0.7875

In [32]:
reg.intercept_

array([-0.16194327])

In [33]:
reg.coef_

array([[ 2.15384942,  0.31354971,  1.6346133 ,  1.45789773,  0.07952414,
         0.73348193, -0.31934949,  0.25578291,  0.11039357,  0.48323468,
        -0.3960797 ]])

In [34]:
featureName = unscaledInputs.columns.values

In [35]:
summaryTable = pd.DataFrame(columns=['Feature name'], data= featureName)
summaryTable['Coefficient'] = np.transpose(reg.coef_)
summaryTable

,Feature name,Coefficient
0,Reason 1,2.153849
1,Reason 2,0.313550
2,Reason 3,1.634613
3,Reason 4,1.457898
4,Month Value,0.079524
5,Transportation Expense,0.733482
6,Age,-0.319349
7,Body Mass Index,0.255783
8,Education,0.110394
9,Children,0.483235


In [36]:
summaryTable.index = summaryTable.index+1
summaryTable.loc[0] = ['Intercept', reg.intercept_[0]]
summaryTable = summaryTable.sort_index()
summaryTable

,Feature name,Coefficient
0,Intercept,-0.161943
1,Reason 1,2.153849
2,Reason 2,0.313550
3,Reason 3,1.634613
4,Reason 4,1.457898
5,Month Value,0.079524
6,Transportation Expense,0.733482
7,Age,-0.319349
8,Body Mass Index,0.255783
9,Education,0.110394


In [37]:
summaryTable['Odds Ratio'] = np.exp(summaryTable.Coefficient)

In [38]:
summaryTable.sort_values('Odds Ratio', ascending = False)

,Feature name,Coefficient,Odds Ratio
1,Reason 1,2.153849,8.617969
3,Reason 3,1.634613,5.127475
4,Reason 4,1.457898,4.296917
6,Transportation Expense,0.733482,2.082318
10,Children,0.483235,1.621310
2,Reason 2,0.313550,1.368273
8,Body Mass Index,0.255783,1.291472
9,Education,0.110394,1.116717
5,Month Value,0.079524,1.082772
0,Intercept,-0.161943,0.850489


In [39]:
reg.score(xTest, yTest)

0.7214285714285714

In [40]:
predictedProba = reg.predict_proba(xTest)
predictedProba

array([[0.82653247, 0.17346753],
       [0.79260388, 0.20739612],
       [0.36391742, 0.63608258],
       [0.25485445, 0.74514555],
       [0.57092378, 0.42907622],
       [0.69799367, 0.30200633],
       [0.75700861, 0.24299139],
       [0.33544071, 0.66455929],
       [0.73040859, 0.26959141],
       [0.34052086, 0.65947914],
       [0.83157844, 0.16842156],
       [0.68833617, 0.31166383],
       [0.23795847, 0.76204153],
       [0.23795847, 0.76204153],
       [0.3559261 , 0.6440739 ],
       [0.993279  , 0.006721  ],
       [0.81847297, 0.18152703],
       [0.33806569, 0.66193431],
       [0.36027403, 0.63972597],
       [0.70275821, 0.29724179],
       [0.23539092, 0.76460908],
       [0.87846031, 0.12153969],
       [0.26024108, 0.73975892],
       [0.16762955, 0.83237045],
       [0.03077571, 0.96922429],
       [0.0963797 , 0.9036203 ],
       [0.04713582, 0.95286418],
       [0.49641858, 0.50358142],
       [0.88876232, 0.11123768],
       [0.32539473, 0.67460527],
       [0.

In [41]:
import pickle

In [42]:
with open('model', 'wb') as file:
    pickle.dump(reg, file)

In [43]:
with open('scaler', 'wb') as file:
    pickle.dump(absenteeismScaler, file)